In [110]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [111]:
import math
import random
import sys
from copy import deepcopy
from functools import partial

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from ay2.tools import freeze_modules
from ay2.torch.nn import LambdaFunctionModule
from einops import rearrange

In [112]:
from torchvision.transforms import v2

In [113]:
try:
    from ...Aaasist.Aaasist.load_model import get_model as load_AASIST
    from ...WaveLM.wavlm import BaseLine as WavLM
except ImportError:
    sys.path.append("../../Aaasist")
    sys.path.append("../../WaveLM")
    from Aaasist.load_model import get_model as load_AASIST
    from wavlm import BaseLine as WavLM

# 1D models

In [5]:
class WavLM_1D(nn.Module):
    def __init__(
        self,
    ):
        super().__init__()
        self.model1D = WavLM()
        self.n_dim = 768

    def forward(self, x):
        if x.ndim == 3:
            x = x[:, 0, :]
        feature = self.model1D.pretrain_model(x)[self.model1D.pretrain_feat]  # (B, T, 768)
        return feature.mean(1)

In [6]:
model1D = WavLM_1D()
x = torch.randn(2, 1, 48000)
model(x).shape

/home/ay/anaconda3/envs/torch/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at /usr/local/ay_data/0-model_weights/microsoft_wavlm-base were not used when initializing WavLMModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing WavLMModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMModel from the checkpoint of a model that you expect to be exactly identical (initializi

NameError: name 'model' is not defined

## 分解 特征提取过程　

### 使用1D CNN 提取特征


In [ ]:
x = torch.randn(2, 48000)
feat = model1D.pretrain_model.feature_extractor(x)

extract_features = model1D.pretrain_model.feature_extractor(x)
extract_features = extract_features.transpose(1, 2)

# 输出的extract_features其实就是输入的layer norm
hidden_states, extract_features = model1D.pretrain_model.feature_projection(extract_features)

### 使用 Transformer 提取特征　

在transformer中，layers共有12层：

In [ ]:
len(model1D.pretrain_model.encoder.layers)

直接提取：

In [ ]:
encoder_outputs = model1D.pretrain_model.encoder(
    hidden_states,
    attention_mask=None,
    output_attentions=False,
    output_hidden_states=False,
    return_dict=False,
)
hidden_states2 = encoder_outputs[0]

提取的详细过程如下：

In [ ]:
layerdrop = 0.05  ## default is 0.05
position_embeddings = model1D.pretrain_model.encoder.pos_conv_embed(hidden_states)
hidden_states = hidden_states + position_embeddings
hidden_states = model1D.pretrain_model.encoder.layer_norm(hidden_states)
hidden_states = model1D.pretrain_model.encoder.dropout(hidden_states)
position_bias = None
for i, layer in enumerate(model1D.pretrain_model.encoder.layers):
    dropout_probability = np.random.uniform(0, 1)
    skip_the_layer = model1D.pretrain_model.encoder.training and i > 0 and (dropout_probability < layerdrop)
    if not skip_the_layer:
        layer_outputs = layer(
            hidden_states,
            attention_mask=None,
            position_bias=position_bias,
            output_attentions=False,
            index=i,
        )
        hidden_states, position_bias = layer_outputs[:2]

经验证，上述过程和直接提取结果是一样的：

In [ ]:
torch.sum(torch.abs(hidden_states2 - hidden_states))

那么，可以将这12层分组，分成多个stage计算：



In [ ]:
class WavLM_1D(nn.Module):
    def __init__(
        self,
    ):
        super().__init__()
        self.model1D = WavLM()
        self.n_dim = 768

    def forward(self, x):
        if x.ndim == 3:
            x = x[:, 0, :]
        feature = self.model1D.pretrain_model(x)[self.model1D.pretrain_feat]  # (B, T, 768)
        return feature.mean(1)

    def compute_stage1(self, x):
        if x.ndim == 3:
            x = x[:, 0, :]
        feat = self.model1D.pretrain_model.feature_extractor(x)
        extract_features = self.model1D.pretrain_model.feature_extractor(x)
        extract_features = extract_features.transpose(1, 2)
        # 输出的extract_features其实就是输入的layer norm
        hidden_states, extract_features = self.model1D.pretrain_model.feature_projection(extract_features)
        return hidden_states

    def compute_transformer_layers(self, hidden_states, s, e):
        for i in range(s, e):
            layer = self.model1D.pretrain_model.encoder.layers[i]
            dropout_probability = np.random.uniform(0, 1)
            skip_the_layer = (
                self.model1D.pretrain_model.encoder.training
                and i > 0
                and (dropout_probability < self.model1D.pretrain_model.encoder.layerdrop)
            )
            if not skip_the_layer:
                layer_outputs = layer(
                    hidden_states,
                    attention_mask=None,
                    position_bias=self.position_bias,
                    output_attentions=False,
                    index=i,
                )
                hidden_states, self.position_bias = layer_outputs[:2]
        return hidden_states

    def compute_stage2(self, hidden_states):
        position_embeddings = self.model1D.pretrain_model.encoder.pos_conv_embed(hidden_states)
        hidden_states = hidden_states + position_embeddings
        hidden_states = self.model1D.pretrain_model.encoder.layer_norm(hidden_states)
        hidden_states = self.model1D.pretrain_model.encoder.dropout(hidden_states)
        self.position_bias = None
        hidden_states = self.compute_transformer_layers(hidden_states, 0, 3)
        return hidden_states

    def compute_stage3(self, hidden_states):
        hidden_states = self.compute_transformer_layers(hidden_states, 3, 6)
        return hidden_states

    def compute_stage4(self, hidden_states):
        hidden_states = self.compute_transformer_layers(hidden_states, 6, 9)
        return hidden_states

    def compute_latent_feature(self, hidden_states):
        hidden_states = self.compute_transformer_layers(hidden_states, 9, 12)
        hidden_states = hidden_states.mean(1)  # (B, C)
        return hidden_states

In [ ]:
model1D = WavLM_1D()
x = torch.randn(2, 1, 48000)
hidden_states = model1D.compute_stage1(x)
hidden_states = model1D.compute_stage2(hidden_states)
hidden_states = model1D.compute_stage3(hidden_states)
hidden_states = model1D.compute_stage4(hidden_states)
hidden_states = model1D.compute_latent_feature(hidden_states)

经验证，上面的过程分解和直接计算的结果是一样的：

In [ ]:
torch.sum(torch.abs(hidden_states - model1D(x)))

# Feature Fusion　

## Expand：将1D wav特征转换为2D　

In [107]:
class CrossAttention2D(nn.Module):
    def __init__(
        self,
        time_dim,
        spec_dim,
        feature_dim,
        num_heads=4,
        dropout_rate=0.1,
        temperature=1.0,
    ):
        super().__init__()
        self.softmax = nn.Softmax(dim=-1)

        self.conv1 = nn.Conv2d(in_channels=time_dim, out_channels=feature_dim, kernel_size=1)
        self.conv2 = nn.Conv2d(in_channels=spec_dim, out_channels=feature_dim, kernel_size=1)
        self.feature_dim = feature_dim

    def forward(self, waveform, spectrogram):
        query = self.conv1(waveform).permute(0, 2, 3, 1)
        key = self.conv2(spectrogram).permute(0, 2, 3, 1)
        value = spectrogram.permute(0, 2, 3, 1)

        # print(query.shape, key.shape, value.shape, torch.matmul(query, key.transpose(-2, -1)).shape)

        attn_weights = self.softmax(torch.matmul(query, key.transpose(-2, -1)) / (self.feature_dim**0.5))
        out = torch.matmul(attn_weights, value).permute(0, 3, 1, 2)

        return out


class Expand(nn.Module):
    def __init__(self, time_len=149, time_dim=768, spec_height=56, spec_width=56, spec_dim=512):
        super().__init__()

        self.time_len = time_len
        self.time_dim = time_dim
        self.spec_height = spec_height
        self.spec_width = spec_width
        self.spec_dim = spec_dim

        self.conv1 = nn.Conv1d(in_channels=time_len, out_channels=spec_height * spec_width, kernel_size=1)
        self.conv2 = nn.Conv2d(in_channels=time_dim, out_channels=spec_dim, kernel_size=3, padding=1)

        self.layer_norm1 = nn.LayerNorm(time_dim)
        self.layer_norm2 = nn.LayerNorm(spec_dim)
        self.attn = CrossAttention2D(time_dim=time_dim, spec_dim=spec_dim, feature_dim=spec_dim)

    def compute_layernorm(self, feat, layer_norm):
        feat = rearrange(feat, "b c h w -> b h w c")
        feat = layer_norm(feat)
        feat = rearrange(feat, "b h w c -> b c h w")
        return feat

    def forward(self, x, y):
        x = self.conv1(x)  # [B, spec_H * spec_W, time_dim]
        x = rearrange(
            x, "b (h w) c -> b c h w", h=self.spec_height, w=self.spec_width
        )  ## [B, time_dim, spec_H, spec_W]
        res = self.attn(self.compute_layernorm(x, self.layer_norm1), self.compute_layernorm(y, self.layer_norm2)) + y
        return res

In [108]:
module = Expand()

B = 3
waveform = torch.rand((B, 149, 768))  # Replace with actual input
spectrogram = torch.rand((B, 512, 56, 56))  # Replace with actual input

In [109]:
module(waveform, spectrogram).shape

torch.Size([3, 512, 56, 56])

In [104]:
class MultiHeadCrossAttention2D(nn.Module):
    def __init__(
        self,
        time_dim,
        spec_dim,
        feature_dim,
        num_heads=4,
        dropout_rate=0.1,
        temperature=1.0,
    ):
        super().__init__()
        assert feature_dim % num_heads == 0, "Feature dimension must be divisible by number of heads"

        self.num_heads = num_heads
        self.feature_dim = feature_dim
        self.feature_dim_head = feature_dim // num_heads
        self.temperature = temperature

        self.key_linear = nn.Linear(spec_dim, feature_dim)
        self.query_linear = nn.Linear(time_dim, feature_dim)
        # self.value_linear = nn.Linear(spec_dim, feature_dim)

        self.dropout = nn.Dropout(dropout_rate)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, waveform, spectrogram):
        batch_size = waveform.shape[0]

        # Apply linear transformations
        keys = self.key_linear(spectrogram)
        queries = self.query_linear(waveform)
        # values = self.value_linear(spectrogram)
        values = spectrogram

        # print(keys.shape, queries.shape, values.shape)
        
        # Split the last dimension into (heads, depth)
        keys = keys.view(batch_size, -1, self.num_heads, self.feature_dim_head).permute(0, 2, 1, 3)
        queries = queries.view(batch_size, -1, self.num_heads, self.feature_dim_head).permute(0, 2, 1, 3)
        values = values.view(batch_size, -1, self.num_heads, self.feature_dim_head).permute(0, 2, 1, 3)

        keys = self.dropout(keys)
        queries = self.dropout(queries)

        # Perform scaled dot-product attention
        score = torch.matmul(queries, keys.transpose(-2, -1)) / math.sqrt(self.feature_dim_head)
        attn_weights = self.softmax(score)
        out = torch.matmul(attn_weights, values).permute(0, 2, 1, 3)

        # Concatenate heads back to single head dimension
        out = out.contiguous().view(batch_size, -1, self.feature_dim)

        return out


class Expand(nn.Module):
    def __init__(self, time_len=149, time_dim=768, spec_height=56, spec_width=56, spec_dim=512, num_heads=1):
        super().__init__()

        self.time_len = time_len
        self.time_dim = time_dim
        self.spec_height = spec_height
        self.spec_width = spec_width
        self.spec_dim = spec_dim

        self.attn = MultiHeadCrossAttention2D(time_dim=time_dim, spec_dim=spec_dim, feature_dim=spec_dim, num_heads=num_heads)
        self.norm_wave = nn.LayerNorm(time_dim)
        self.norm_spec = nn.LayerNorm(spec_dim)

        self.conv_wave = nn.Conv1d(in_channels=time_len, out_channels=spec_height * spec_width, kernel_size=1)
        self.conv_spec = nn.Conv2d(in_channels=time_dim, out_channels=spec_dim, kernel_size=3, padding=1)

        self.positional_encoding_wave = nn.Parameter(torch.randn(1, spec_height * spec_width, time_dim), requires_grad=True)
        self.positional_encoding_spec = nn.Parameter(torch.randn(1, spec_dim, spec_height, spec_width), requires_grad=True)
        self.dropout = nn.Dropout(p=0.1)

    def compute_layernorm(self, feat, layer_norm):
        feat = rearrange(feat, "b c h w -> b h w c")
        feat = layer_norm(feat)
        feat = rearrange(feat, "b h w c -> b c h w")
        return feat

    def forward(self, x, y):
        """
        Args:
            x: the waveform feature of size (B, time_len, time_dim)
            y: the spectrogram feature of size (B, spec_dim, spec_height, spec_width)

        Returns:
            y + attn(x, y)
        """
        
        x = self.conv_wave(x)  # [B, spec_H * spec_W, time_dim]
        norm_x = self.norm_wave(x + self.positional_encoding_wave)  # [B, spec_H * spec_W, time_dim]

        norm_y = self.compute_layernorm(y + self.positional_encoding_spec, self.norm_spec)
        norm_y = rearrange(norm_y, "b c h w -> b (h w) c")

        res = self.attn(norm_x, norm_y)
        # res = norm_y
        res = self.dropout(res)
        res = rearrange(res, "b (h w) c -> b c h w", h=self.spec_height, w=self.spec_width)

        return res + y

In [105]:
module = Expand()

B = 3
waveform = torch.rand((B, 149, 768))  # Replace with actual input
spectrogram = torch.rand((B, 512, 56, 56))  # Replace with actual input

In [106]:
module(waveform, spectrogram).shape

torch.Size([3, 512, 56, 56])

In [103]:
m = nn.MultiheadAttention(768, num_heads=4, batch_first=True)
x = torch.randn(3, 149, 768)
y = torch.randn(3, 3136, 768)
res, _ = m(x,y, y, need_weights=False)
print(res.shape)

torch.Size([3, 149, 768])


### Squeeze：将2D spec squeeze 到1D　

In [7]:
class CrossAttention1D(nn.Module):
    def __init__(self, time_dim, spec_dim, feature_dim):
        super().__init__()
        self.softmax = nn.Softmax(dim=-1)

        self.linear1 = nn.Linear(time_dim, feature_dim)
        self.linear2 = nn.Linear(spec_dim, feature_dim)
        self.feature_dim = feature_dim

    def forward(self, waveform, spectrogram):
        """
        Args:
            waveform: (B, time_len, time_dim)
            spectrogram: (B, time_len, spec_dim)

        """
        key = self.linear1(waveform)  ##  (B, time_len, feature_dim)
        query = self.linear2(spectrogram)  ##  (B, time_len, feature_dim)
        value = waveform

        attn_weights = self.softmax(torch.matmul(query, key.transpose(-2, -1)) / (self.feature_dim**0.5))
        out = torch.matmul(attn_weights, value)  ##  (B, time_len, feature_dim)

        return out

In [8]:
class Squeeze(nn.Module):
    def __init__(self, time_len=149, time_dim=768, spec_height=56, spec_width=56, spec_dim=512):
        super().__init__()

        self.time_len = time_len
        self.time_dim = time_dim
        self.spec_height = spec_height
        self.spec_width = spec_width
        self.spec_dim = spec_dim

        ### used to convert spec into waveform
        self.linear = nn.Linear(spec_height * spec_width, time_len)

        self.layer_norm1 = nn.LayerNorm(time_dim)
        self.layer_norm2 = nn.LayerNorm(spec_dim)
        self.attn = CrossAttention1D(time_dim=time_dim, spec_dim=spec_dim, feature_dim=spec_dim)

    def forward(self, x, y):
        y = rearrange(y, "b c h w -> b c (h w)")
        y = self.linear(y)  ### # [B, time_len, spec_dim]
        y = rearrange(y, "b c l -> b l c")

        res = self.attn(self.layer_norm1(x), self.layer_norm2(y)) + x
        return res

In [9]:
module = Squeeze()

B = 3
waveform = torch.rand((B, 149, 768))  # Replace with actual input
spectrogram = torch.rand((B, 512, 56, 56))  # Replace with actual input


module(waveform, spectrogram).shape

torch.Size([3, 149, 768])

### 增强　

使用chatgpt改进的code

In [ ]:
class CrossAttention1D(nn.Module):
    def __init__(
        self,
        time_dim,
        spec_dim,
        feature_dim,
        dropout_rate=0.1,
        temperature=1.0,
        num_heads=1,  # Placeholder for multi-head attention
    ):
        super().__init__()
        self.softmax = nn.Softmax(dim=-1)
        self.temperature = temperature

        self.linear1 = nn.Linear(time_dim, feature_dim)
        self.linear2 = nn.Linear(spec_dim, feature_dim)
        self.dropout = nn.Dropout(dropout_rate)

        self.norm_key = nn.LayerNorm(feature_dim)
        self.norm_query = nn.LayerNorm(feature_dim)

        # Placeholder attributes for multi-head attention
        self.num_heads = num_heads
        self.feature_dim_head = feature_dim // num_heads

        self.feature_dim = feature_dim

    def forward(self, waveform, spectrogram):
        key = self.norm_key(self.linear1(waveform))
        query = self.norm_query(self.linear2(spectrogram))
        value = waveform  # Consider operation on value for multi-head attention

        key = self.dropout(key)
        query = self.dropout(query)

        attn_weights = self.softmax(
            torch.matmul(query, key.transpose(-2, -1)) / (self.temperature * math.sqrt(self.feature_dim_head))
        )
        out = torch.matmul(attn_weights, value)

        return out

In [ ]:
class MultiHeadCrossAttention1D(nn.Module):
    def __init__(
        self,
        time_dim,
        spec_dim,
        feature_dim,
        num_heads=4,
        dropout_rate=0.1,
        temperature=1.0,
    ):
        super().__init__()
        assert feature_dim % num_heads == 0, "Feature dimension must be divisible by number of heads"

        self.num_heads = num_heads
        self.feature_dim = feature_dim
        self.feature_dim_head = feature_dim // num_heads
        self.temperature = temperature

        self.key_linear = nn.Linear(time_dim, feature_dim)
        self.query_linear = nn.Linear(spec_dim, feature_dim)
        self.value_linear = nn.Linear(time_dim, feature_dim)

        self.dropout = nn.Dropout(dropout_rate)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, waveform, spectrogram):
        batch_size = waveform.shape[0]

        # Apply linear transformations
        keys = self.key_linear(waveform)
        queries = self.query_linear(spectrogram)
        values = self.value_linear(waveform)

        # Split the last dimension into (heads, depth)
        keys = keys.view(batch_size, -1, self.num_heads, self.feature_dim_head).permute(0, 2, 1, 3)
        queries = queries.view(batch_size, -1, self.num_heads, self.feature_dim_head).permute(0, 2, 1, 3)
        values = values.view(batch_size, -1, self.num_heads, self.feature_dim_head).permute(0, 2, 1, 3)

        keys = self.dropout(keys)
        queries = self.dropout(queries)

        # Perform scaled dot-product attention
        score = torch.matmul(queries, keys.transpose(-2, -1)) / math.sqrt(self.feature_dim_head)
        attn_weights = self.softmax(score)
        out = torch.matmul(attn_weights, values).permute(0, 2, 1, 3)

        # Concatenate heads back to single head dimension
        out = out.contiguous().view(batch_size, -1, self.feature_dim)

        return out


class Squeeze(nn.Module):
    def __init__(self, time_len=149, time_dim=768, spec_height=56, spec_width=56, spec_dim=512, dropout_rate=0.1):
        super().__init__()

        self.linear = nn.Linear(spec_height * spec_width, time_len)
        self.layer_norm1 = nn.LayerNorm(time_dim)
        self.layer_norm2 = nn.LayerNorm(spec_dim)
        self.dropout = nn.Dropout(dropout_rate)

        # Add placeholders for positional encoding
        self.positional_encoding_wave = nn.Parameter(torch.randn(1, time_len, time_dim), requires_grad=True)
        self.positional_encoding_spec = nn.Parameter(torch.randn(1, time_len, spec_dim), requires_grad=True)

        self.attn = MultiHeadCrossAttention1D(
            time_dim=time_dim, spec_dim=spec_dim, feature_dim=time_dim, dropout_rate=dropout_rate
        )

    def forward(self, x, y):
        y = rearrange(y, "b c h w -> b c (h w)")
        y = self.linear(y)
        y = rearrange(y, "b c l -> b l c")

        # Apply positional encodings
        norm_x = self.layer_norm1(x + self.positional_encoding_wave)
        norm_y = self.layer_norm2(y + self.positional_encoding_spec)

        res = self.attn(norm_x, norm_y)
        res = self.dropout(res)
        res += x  # Residual connection

        return res

In [30]:
module = Squeeze()

B = 3
waveform = torch.rand((B, 149, 768))  # Replace with actual input
spectrogram = torch.rand((B, 512, 56, 56))  # Replace with actual input


module(waveform, spectrogram).shape

torch.Size([3, 149, 768])

# 重建损失　

In [ ]:
import torch
import torch.nn as nn


class WaveformToSpectrogram(nn.Module):
    def __init__(self):
        super(WaveformToSpectrogram, self).__init__()
        self.conv1 = nn.Conv1d(149, 224, kernel_size=3, stride=1, padding=1)  # Adjust the channels
        self.pool = nn.AdaptiveAvgPool1d(224)  # Adjust time dimension to match spectrogram
        
    def forward(self, x):
        x = self.conv1(x)  # x: [B, 224, 768]
        x = self.pool(x)  # x: [B, 224, 224]
        return x

class TimeFrequencyReconstructionLoss:

    def __init__(self):
        self.module = WaveformToSpectrogram()
        self.loss = nn.MSELoss()　

    def forward(self, x, y):
        x = self.module(x)
        loss = self.loss(x, y[:, 0, :, :])
        return loss


B = 3
# Assuming the input waveform tensor
waveform_input = torch.rand((B, 149, 768))  # Example input

# Model and forward pass
model = WaveformToSpectrogram()
output = model(waveform_input)
output.shape

# Gated fusion

In [124]:
import torch
import torch.nn as nn

class GatedFusionLayer(nn.Module):
    def __init__(self, waveform_dim, spectrogram_dim, combined_dim):
        super(GatedFusionLayer, self).__init__()

        self.proj = nn.Linear(waveform_dim, spectrogram_dim)
        
        self.fc = nn.Linear(spectrogram_dim * 2, combined_dim)
        self.gate_fc = nn.Linear(combined_dim, 2)

    def forward(self, waveform_features, spectrogram_features):
        waveform_features = self.proj(waveform_features)
        # Flatten features if needed
        waveform_features_flat = waveform_features.view(waveform_features.size(0), -1)
        spectrogram_features_flat = spectrogram_features.view(spectrogram_features.size(0), -1)

        # Concatenate features
        combined_features = torch.cat([waveform_features_flat, spectrogram_features_flat], dim=-1)

        # Pass through fully connected layer
        combined_features = torch.relu(self.fc(combined_features))

        # Compute gate weights
        gate = torch.sigmoid(self.gate_fc(combined_features))

        # Apply weights to the features
        weight_waveform, weight_spectrogram = gate[:, 0], gate[:, 1]
        weight_waveform = weight_waveform.view(-1, 1)
        weight_spectrogram = weight_spectrogram.view(-1, 1)

        weighted_waveform_features = waveform_features * weight_waveform
        weighted_spectrogram_features = spectrogram_features * weight_spectrogram

        # Fuse the features
        fused_features = weighted_waveform_features + weighted_spectrogram_features

        return fused_features

In [129]:
x = torch.randn(3, 768)
y = torch.randn(3, 512)
m = GatedFusionLayer(768, 512, 1024)
m(x, y).shape

torch.Size([3, 512])